In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
train_df = pd.read_csv('../input/spaceship-titanic/train.csv')
test_df = pd.read_csv('../input/spaceship-titanic/test.csv')
train_df.columns


In [7]:
X = train_df.drop(["PassengerId","Name","Transported","VRDeck","HomePlanet"],axis = 1)
y = train_df['Transported']
XT= test_df.drop(["PassengerId","Name","VRDeck","HomePlanet"],axis = 1)


In [8]:
X


In [9]:
!pip install pandas-profiling


In [ ]:
import pandas_profiling as pp
pp.ProfileReport(train_df,title="Pandas Profiling Report") 

In [ ]:
X['Destination'].unique()

In [ ]:
X.select_dtypes(['object'])
X['CryoSleep'] = X['CryoSleep'].fillna(X['CryoSleep'].mode()[0])
X['Cabin'] = X['Cabin'].fillna(X['Cabin'].mode()[0])
X['Destination'] = X['Destination'].fillna(X['Destination'].mode()[0])
X['VIP'] = X['VIP'].fillna(X['VIP'].mode()[0])
XT['CryoSleep'] = XT['CryoSleep'].fillna(XT['CryoSleep'].mode()[0])
XT['Cabin'] = XT['Cabin'].fillna(XT['Cabin'].mode()[0])
XT['Destination'] = XT['Destination'].fillna(XT['Destination'].mode()[0])
XT['VIP'] = XT['VIP'].fillna(XT['VIP'].mode()[0])

In [ ]:
X.dtypes
X['Age'] = X['Age'].fillna(X['Age'].median())
X['RoomService'] = X['RoomService'].fillna(X['RoomService'].mean())
X['FoodCourt'] = X['FoodCourt'].fillna(X['FoodCourt'].mean())
X['ShoppingMall'] = X['ShoppingMall'].fillna(X['ShoppingMall'].mean())
X['Spa'] = X['Spa'].fillna(X['Spa'].mean())

In [ ]:
XT['Age'] = XT['Age'].fillna(XT['Age'].median())
XT['RoomService'] = XT['RoomService'].fillna(XT['RoomService'].mean())
XT['FoodCourt'] = XT['FoodCourt'].fillna(XT['FoodCourt'].mean())
XT['ShoppingMall'] = XT['ShoppingMall'].fillna(XT['ShoppingMall'].mean())
XT['Spa'] = XT['Spa'].fillna(XT['Spa'].mean())

In [ ]:
X.isnull().sum()


In [ ]:

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
X['CryoSleep']= label_encoder.fit_transform(X['CryoSleep'])
X['Destination']= label_encoder.fit_transform(X['Destination'])
X['Cabin']= label_encoder.fit_transform(X['Cabin'])

In [ ]:
XT['CryoSleep']= label_encoder.fit_transform(XT['CryoSleep'])
XT['Destination']= label_encoder.fit_transform(XT['Destination'])
XT['Cabin']= label_encoder.fit_transform(XT['Cabin'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

In [ ]:
X_train

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#classifier = RandomForestClassifier(n_estimators = 100, random_state=0)
#classifier.fit(X,y)




In [ ]:
from catboost import CatBoostClassifier
from sklearn.feature_selection import SequentialFeatureSelector

model_fs = CatBoostClassifier(verbose=False)
sf = SequentialFeatureSelector(model_fs, scoring='accuracy', direction = 'backward')
sf.fit(X,y)

In [ ]:
best_features = list(sf.get_feature_names_out())
best_features
model = CatBoostClassifier(verbose=False, eval_metric='Accuracy')
model.fit(X[best_features], y)
prediction = model.predict(test_df[best_features])

In [ ]:
preds = model.predict(X_test)
#preds_test = classifier.predict(XT)


In [ ]:
prediction

In [ ]:
 from sklearn import metrics
 #confusion = metrics.confusion_matrix(y_test, prediction)
#confusion.ravel() 

In [ ]:
 accuracy = metrics.accuracy_score(y_test, preds)
 accuracy 

In [ ]:
result = pd.DataFrame()
result['PassengerId'] = test_df.PassengerId
result['Transported'] = prediction



In [ ]:
result.to_csv("spaceship.csv",index=False)